In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

from torchvision.datasets import ImageFolder
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.image as mpimg

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(), #0~255 -> 0~1
    transforms.Normalize(  #0~1 -> -1~1   (x-mean)/std
    mean=[0.5,0.5,0.5],
    std=[0.5,0.5,0.5]
)
])


In [ ]:
dataset = ImageFolder("/home/0753115/data/DL_HW3/",transform=preprocess)

In [ ]:
train_data=torch.utils.data.DataLoader(dataset, batch_size=128, num_workers=0, shuffle=True,drop_last=True)

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    print('classname:', classname)

    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


In [ ]:
# Discriminator
class Discriminator(nn.Module):
    def __init__(self, inputSize, hiddenSize):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(inputSize, hiddenSize, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(hiddenSize, hiddenSize*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hiddenSize*2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(hiddenSize*2, hiddenSize*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hiddenSize*4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(hiddenSize*4, hiddenSize*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hiddenSize*8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(hiddenSize*8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid())

    def forward(self, input):
        return self.main(input)

In [ ]:
# Generator
class Generator(nn.Module):
    def __init__(self, inputSize, hiddenSize, outputSize):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(inputSize, hiddenSize*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(hiddenSize*8),
            nn.ReLU(True),

            nn.ConvTranspose2d(hiddenSize*8, hiddenSize*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hiddenSize*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(hiddenSize*4, hiddenSize*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hiddenSize*2),
            nn.ReLU(True),

            nn.ConvTranspose2d(hiddenSize*2, hiddenSize, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hiddenSize),
            nn.ReLU(True),

            nn.ConvTranspose2d(hiddenSize, outputSize, 4, 2, 1, bias=False),
            nn.Tanh())

    def forward(self, input):
        return self.main(input)

In [ ]:
batch_size = 1024
image_size = 64
G_out_D_in = 3
G_in = 100
G_hidden = 64
D_hidden = 64

lr = 0.001
beta1 = 0.5

In [ ]:
# Create the generator
netG = Generator(G_in, G_hidden, G_out_D_in)
netG.apply(weights_init)
print(netG)

# Create the discriminator
netD = Discriminator(G_out_D_in, D_hidden)
netD.apply(weights_init)
print(netD)

# Loss fuG_out_D_intion
criterion = nn.BCELoss()
fixed_noise = torch.randn(64, G_in, 1, 1)

real_label = 1
fake_label = 0
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

img_list = []
G_losses = []
D_losses = []
iters = 0

In [ ]:
epochs = 5
G_loss = []
G_temp=[]
D_loss = []
D_temp=[]

for epoch in range(epochs):
    for i, data in enumerate(train_data, 0):
        # Update D network
        optimizerD.zero_grad()
        
        #D判斷real images
        inputs = data[0]
        b_size = inputs.size(0)
        label = torch.full((b_size,), real_label)
        output = netD(inputs).view(-1)

        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()  #越接近1越好

        #D判斷fake images        
        noise = torch.randn(b_size, G_in, 1, 1)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach()).view(-1)

        errD_fake = criterion(output, label)
        errD_fake.backward()

        D_G_z1 = output.mean().item()  #越接近0越表示Discriminator越強
        errD = errD_real + errD_fake
        if i%2==0:
            optimizerD.step()

        
        # Update G network
        optimizerG.zero_grad()
        label.fill_(real_label)
        output = netD(fake).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()  #越接近1表示Discrminator越強
        optimizerG.step()

        G_temp.append(errG.item())
        D_temp.append(errD.item())
        
    G_loss.append(np.mean(G_temp))
    D_loss.append(np.mean(D_temp))
    
    print('[%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x)' % (epoch, epochs, np.mean(D_temp), np.mean(G_temp)))

    
#     torch.save(netD, 'netD.pkl')
#     torch.save(netG, 'netG.pkl')

In [ ]:
plt.plot(range(len(G_loss)),G_loss,'b',range(len(D_loss)),D_loss,'g')

In [ ]:
plt.plot(range(len(G_temp)),G_temp,'b',range(len(D_temp)),D_temp,'g')

In [ ]:
# 畫出input圖片及模型產生的結果
import torchvision.utils as vutils

img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

Image.fromarray(np.transpose(((vutils.make_grid(inputs, padding=2, normalize=False).data.numpy()+1.0)/2.0*255).astype(np.uint8),(1,2,0)))

Image.fromarray(np.transpose(((vutils.make_grid(fake, padding=2, normalize=False).data.numpy()+1.0)/2.0*255).astype(np.uint8),(1,2,0)))